In [0]:
!pip install -q findspark 

In [71]:
!pip install pyspark

In [72]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [74]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)


In [0]:
import findspark
import pyspark
from pyspark.sql import *
import pandas as pd

In [0]:
MAX_MEMORY = '6g'
spark_session = SparkSession.builder \
    .master("local") \
    .appName("Task 3") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY)\
    .getOrCreate()

In [77]:
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/train.csv", delimiter = ",")
df.head(5)

,movie_id,movie_name,plot,genre
0,23890098,Taxi Blues,"Shlykov, a hard-working taxi driver and Lyosha...","['World cinema', 'Drama']"
1,31186339,The Hunger Games,The nation of Panem consists of a wealthy Capi...,"['Action/Adventure', 'Action', 'Science Fictio..."
2,20663735,Narasimham,Poovalli Induchoodan is sentenced for six yea...,"['Musical', 'Action', 'Drama']"
3,2231378,The Lemon Drop Kid,"The Lemon Drop Kid , a New York City swindler,...",['Comedy']
4,595909,A Cry in the Dark,Seventh-day Adventist Church pastor Michael Ch...,"['Crime Fiction', 'World cinema', 'Drama']"


In [78]:
sdf1 = spark_session.createDataFrame(df)
sdf1.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [0]:
from pyspark.sql.functions import *
new= sdf1.withColumn('plot', regexp_replace('plot', '[{.*}]', ''))\
            .withColumn('plot', regexp_replace('plot', 'plot', '')) \
            .withColumn('plot', regexp_replace('plot', '[\(\)]',''))\
            .withColumn('plot', regexp_replace('plot', '[\[\]]', '')) \
            .withColumn('genre', regexp_replace('genre', '[\[\]]', '')) \
            .withColumn('plot', regexp_replace('plot',',',''))\
            .withColumn('genre', regexp_replace('genre', '[\']', ''))\
            .withColumn('genre', regexp_replace('genre',' ', ''))

In [80]:
sdf1 = new
#sdf1 = new1
sdf1.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov a hard-wo...|   Worldcinema,Drama|
|31186339|    The Hunger Games|The nation of Pan...|Action/Adventure,...|
|20663735|          Narasimham|Poovalli Induchoo...|Musical,Action,Drama|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|              Comedy|
|  595909|   A Cry in the Dark|Seventh-day Adven...|CrimeFiction,Worl...|
| 5272176|            End Game|The president is ...|Action/Adventure,...|
| 1952976|          Dark Water| The film opens i...|Thriller,Drama,Ho...|
|24225279|                Sing|The story begins ...|               Drama|
| 2462689|       Meet John Doe|Infuriated at bei...|Black-and-white,C...|
|20532852|Destination Meatball|A line of people ...|Animation,ShortFi...|
|15401493|    Husband for Hire|Lola  a

In [81]:
import pyspark.sql.types as types
sdf2 = spark_session.read.csv("/content/gdrive/My Drive/Colab Notebooks/mapping.csv", sep = ',', escape='"', enforceSchema='true', header = 'true')
sdf2.show()

+---+----------------+
|_c0|               0|
+---+----------------+
|  0|           Drama|
|  1|          Comedy|
|  2|    Romance Film|
|  3|        Thriller|
|  4|          Action|
|  5|    World cinema|
|  6|   Crime Fiction|
|  7|          Horror|
|  8| Black-and-white|
|  9|           Indie|
| 10|Action/Adventure|
| 11|       Adventure|
| 12|     Family Film|
| 13|      Short Film|
| 14|  Romantic drama|
| 15|       Animation|
| 16|         Musical|
| 17| Science Fiction|
| 18|         Mystery|
| 19| Romantic comedy|
+---+----------------+



In [82]:
sdf2 = sdf2.withColumnRenamed('_c0','id')\
    .withColumnRenamed('0', 'genres')
sdf2.show()

+---+----------------+
| id|          genres|
+---+----------------+
|  0|           Drama|
|  1|          Comedy|
|  2|    Romance Film|
|  3|        Thriller|
|  4|          Action|
|  5|    World cinema|
|  6|   Crime Fiction|
|  7|          Horror|
|  8| Black-and-white|
|  9|           Indie|
| 10|Action/Adventure|
| 11|       Adventure|
| 12|     Family Film|
| 13|      Short Film|
| 14|  Romantic drama|
| 15|       Animation|
| 16|         Musical|
| 17| Science Fiction|
| 18|         Mystery|
| 19| Romantic comedy|
+---+----------------+



In [83]:
sdf2 = sdf2.withColumn('genres', regexp_replace('genres',' ',''))
sdf2.show()

+---+----------------+
| id|          genres|
+---+----------------+
|  0|           Drama|
|  1|          Comedy|
|  2|     RomanceFilm|
|  3|        Thriller|
|  4|          Action|
|  5|     Worldcinema|
|  6|    CrimeFiction|
|  7|          Horror|
|  8| Black-and-white|
|  9|           Indie|
| 10|Action/Adventure|
| 11|       Adventure|
| 12|      FamilyFilm|
| 13|       ShortFilm|
| 14|   Romanticdrama|
| 15|       Animation|
| 16|         Musical|
| 17|  ScienceFiction|
| 18|         Mystery|
| 19|  Romanticcomedy|
+---+----------------+



In [84]:
vals = sdf2.select('genres').collect()
print(vals[0])

Row(genres='Drama')


In [85]:
genre_list = []
for i in range(len(vals)):
    genre_list.append(vals[i].asDict()['genres'])
genre_list

['Drama',
 'Comedy',
 'RomanceFilm',
 'Thriller',
 'Action',
 'Worldcinema',
 'CrimeFiction',
 'Horror',
 'Black-and-white',
 'Indie',
 'Action/Adventure',
 'Adventure',
 'FamilyFilm',
 'ShortFilm',
 'Romanticdrama',
 'Animation',
 'Musical',
 'ScienceFiction',
 'Mystery',
 'Romanticcomedy']

In [86]:
train_genre = sdf1.select('genre').collect()
v = train_genre[0].asDict()['genre']
v = v.split(",")
print(v)

['Worldcinema', 'Drama']


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import numpy as np

def generate_one_hot_encoding(genre):
    v = genre.split(",")
    encoding = np.zeros(len(genre_list))
    for i in range(len(v)):
        for j in range(len(genre_list)):
            if v[i] == genre_list[j]:
                encoding[j] = 1 
            else:
                pass
    
    
    encoding = encoding.tolist()
    return encoding

In [88]:
import pyspark.sql.types as types
udffunc = udf(generate_one_hot_encoding, types.ArrayType(FloatType()))
new_sdf = sdf1.withColumn('encoding', udffunc('genre')) 
new_sdf.show()

+--------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|            encoding|
+--------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov a hard-wo...|   Worldcinema,Drama|[1.0, 0.0, 0.0, 0...|
|31186339|    The Hunger Games|The nation of Pan...|Action/Adventure,...|[1.0, 0.0, 0.0, 0...|
|20663735|          Narasimham|Poovalli Induchoo...|Musical,Action,Drama|[1.0, 0.0, 0.0, 0...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|              Comedy|[0.0, 1.0, 0.0, 0...|
|  595909|   A Cry in the Dark|Seventh-day Adven...|CrimeFiction,Worl...|[1.0, 0.0, 0.0, 0...|
| 5272176|            End Game|The president is ...|Action/Adventure,...|[1.0, 0.0, 0.0, 1...|
| 1952976|          Dark Water| The film opens i...|Thriller,Drama,Ho...|[1.0, 0.0, 0.0, 1...|
|24225279|                Sing|The story begins ..

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
tokenizer = Tokenizer(inputCol='plot', outputCol='plottokens')
new_sdf = tokenizer.transform(new_sdf)
remover = StopWordsRemover(inputCol = 'plottokens', outputCol = 'plottokensclean')
new_sdf = remover.transform(new_sdf)

In [0]:
new_sdf = new_sdf.drop('plottokens')

In [91]:
new_sdf.schema
new_sdf.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|            encoding|     plottokensclean|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov a hard-wo...|   Worldcinema,Drama|[1.0, 0.0, 0.0, 0...|[shlykov, hard-wo...|
|31186339|    The Hunger Games|The nation of Pan...|Action/Adventure,...|[1.0, 0.0, 0.0, 0...|[nation, panem, c...|
|20663735|          Narasimham|Poovalli Induchoo...|Musical,Action,Drama|[1.0, 0.0, 0.0, 0...|[poovalli, induch...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|              Comedy|[0.0, 1.0, 0.0, 0...|[lemon, drop, kid...|
|  595909|   A Cry in the Dark|Seventh-day Adven...|CrimeFiction,Worl...|[1.0, 0.0, 0.0, 0...|[seventh-day, adv...|
| 5272176|            End Game|The president is ...|Action/Adventure,...

In [92]:
sdf3 = spark_session.read.csv("/content/gdrive/My Drive/Colab Notebooks/test.csv", sep = ',', escape='"', enforceSchema='true', header = 'true')
sdf3.show()

+--------+--------------------+--------------------+
|movie_id|          movie_name|                plot|
+--------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|
|29062594|A la salida nos v...|A group of teenag...|
| 9252321|   Come Back, Africa|This story of a Z...|
|13455076|       A Merry Mixup|The Stooges play ...|
|24165951|        Getting Even|A soldier-of-fort...|
| 1925869|  River of No Return|Set in the Northw...|
|10799612|          Amici miei|Like in many othe...|
|28238240|Mickey's Big Game...|Mickey and the Sc...|
|17124781|The Good, the Bad...|In the desert wil...|
|28207941|    The Dancing Fool|Bimbo and Koko ar...|
|19174305|              Tahaan|Tahaan  lives wit...|
|18392317|     Mysterious Mose|Betty is startled...|
|34420857|Kelviyum Naane Pa...|Nirmal ([[Karthik...|
| 4039635|   First on the Moon|A group of journa...|
| 8034072|  Journey of a Woman|Vaibhavari Sahay,...|
| 4016437|     Sophie's Choice|In 1947, the mo

In [93]:
new3 = sdf3.withColumn('plot', regexp_replace('plot', '[{.*}]', ''))\
            .withColumn('plot', regexp_replace('plot', 'plot', '')) \
            .withColumn('plot', regexp_replace('plot', '[\(\)]',''))\
            .withColumn('plot', regexp_replace('plot', '[\[\]]',''))
sdf3 = new3
sdf3.show()

+--------+--------------------+--------------------+
|movie_id|          movie_name|                plot|
+--------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|
|29062594|A la salida nos v...|A group of teenag...|
| 9252321|   Come Back, Africa|This story of a Z...|
|13455076|       A Merry Mixup|The Stooges play ...|
|24165951|        Getting Even|A soldier-of-fort...|
| 1925869|  River of No Return|Set in the Northw...|
|10799612|          Amici miei|Like in many othe...|
|28238240|Mickey's Big Game...|Mickey and the Sc...|
|17124781|The Good, the Bad...|In the desert wil...|
|28207941|    The Dancing Fool|Bimbo and Koko ar...|
|19174305|              Tahaan|Tahaan  lives wit...|
|18392317|     Mysterious Mose|Betty is startled...|
|34420857|Kelviyum Naane Pa...|Nirmal Karthik  t...|
| 4039635|   First on the Moon|A group of journa...|
| 8034072|  Journey of a Woman|Vaibhavari Sahay,...|
| 4016437|     Sophie's Choice|In 1947, the mo

In [94]:
sdf3 = tokenizer.transform(sdf3)
sdf3 = remover.transform(sdf3)
sdf3.show()

+--------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|          plottokens|     plottokensclean|
+--------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[the, film, is, b...|[film, based, eve...|
|29062594|A la salida nos v...|A group of teenag...|[a, group, of, te...|[group, teenagers...|
| 9252321|   Come Back, Africa|This story of a Z...|[this, story, of,...|[story, zulu, fam...|
|13455076|       A Merry Mixup|The Stooges play ...|[the, stooges, pl...|[stooges, play, t...|
|24165951|        Getting Even|A soldier-of-fort...|[a, soldier-of-fo...|[soldier-of-fortu...|
| 1925869|  River of No Return|Set in the Northw...|[set, in, the, no...|[set, northwester...|
|10799612|          Amici miei|Like in many othe...|[like, in, many, ...|[like, many, moni...|
|28238240|Mickey's Big Game...|Mickey and the Sc..

In [95]:
sdf3 = sdf3.drop('plottokens')
sdf3.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|     plottokensclean|
+--------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[film, based, eve...|
|29062594|A la salida nos v...|A group of teenag...|[group, teenagers...|
| 9252321|   Come Back, Africa|This story of a Z...|[story, zulu, fam...|
|13455076|       A Merry Mixup|The Stooges play ...|[stooges, play, t...|
|24165951|        Getting Even|A soldier-of-fort...|[soldier-of-fortu...|
| 1925869|  River of No Return|Set in the Northw...|[set, northwester...|
|10799612|          Amici miei|Like in many othe...|[like, many, moni...|
|28238240|Mickey's Big Game...|Mickey and the Sc...|[mickey, scorpion...|
|17124781|The Good, the Bad...|In the desert wil...|[desert, wilderne...|
|28207941|    The Dancing Fool|Bimbo and Koko ar...|[bimbo, koko, sig...|
|19174305|              Tahaan|Tahaan 

In [0]:
for i in range(len(genre_list)):
    new_sdf = new_sdf.withColumn("col{}".format(i),new_sdf['encoding'].getItem(i))

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF

In [0]:
def make_and_fit(sdf, inputcol):
    hT = HashingTF(inputCol=inputcol, outputCol='plotvectors')
    doc2vec_pipeline = Pipeline(stages =[hT] )
    doc2vec_model = doc2vec_pipeline.fit(sdf)
    return doc2vec_model

In [0]:
doc2vec_model = make_and_fit(new_sdf, inputcol="plottokensclean")
new_sdf = doc2vec_model.transform(new_sdf)

In [100]:
new_sdf.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------------------+
|movie_id|          movie_name|                plot|               genre|            encoding|     plottokensclean|col0|col1|col2|col3|col4|col5|col6|col7|col8|col9|col10|col11|col12|col13|col14|col15|col16|col17|col18|col19|         plotvectors|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------------------+
|23890098|          Taxi Blues|Shlykov a hard-wo...|   Worldcinema,Drama|[1.0, 0.0, 0.0, 0...|[shlykov, hard-wo...| 1.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0|(262144,[12751,14...|
|31186339|  

In [0]:
d2v_model = make_and_fit(sdf3, inputcol = "plottokensclean")

In [105]:
sdf3 = d2v_model.transform(sdf3)
sdf3.show()

+--------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|     plottokensclean|         plotvectors|
+--------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[film, based, eve...|(262144,[1728,261...|
|29062594|A la salida nos v...|A group of teenag...|[group, teenagers...|(262144,[6068,191...|
| 9252321|   Come Back, Africa|This story of a Z...|[story, zulu, fam...|(262144,[1598,208...|
|13455076|       A Merry Mixup|The Stooges play ...|[stooges, play, t...|(262144,[6183,943...|
|24165951|        Getting Even|A soldier-of-fort...|[soldier-of-fortu...|(262144,[4098,644...|
| 1925869|  River of No Return|Set in the Northw...|[set, northwester...|(262144,[535,3294...|
|10799612|          Amici miei|Like in many othe...|[like, many, moni...|(262144,[210,5053...|
|28238240|Mickey's Big Game...|Mickey and the Sc..

In [0]:
from pyspark.ml.classification import RandomForestClassifier

In [0]:
def fit_and_classify(sdf, labelcol, featurescol):
    rf_classifier = RandomForestClassifier(labelCol = labelcol, featuresCol= featurescol,numTrees=3,maxDepth=2)
    rf_classifier_pipeline = Pipeline(stages=[rf_classifier])
    rf_model = rf_classifier_pipeline.fit(sdf)
    return rf_model

In [0]:
# Code fragment for testing undersampling
l = []
for i in range(20):
    filter_df = new_sdf.select('movie_id','movie_name', 'plot','genre','plotvectors','col{}'.format(i))
#Dividing the filtered_df to check for the ratio
    df1 = filter_df.filter(filter_df['col{}'.format(i)] == 1.0)
    df2 = filter_df.filter(filter_df['col{}'.format(i)] == 0.0)
    if df2.count() > df1.count():
        ratio = df2.count() / df1.count()
        sampled_df = df2.sample(False, 1/ratio)
        final_df = sampled_df.union(df1)
    else:
        ratio = df1.count() / df2.count()
        sampled_df = df1.sample(False, 1/ratio)
        final_df = sampled_df.union(df2)
    rf_model = fit_and_classify(final_df, labelcol='col{}'.format(i), featurescol='plotvectors')
    predictions = rf_model.transform(sdf3)
    l.append(predictions.select('movie_id', 'prediction'))

In [116]:
len(l)

20

In [0]:
for i in range(len(l)):
    l[i] = l[i].withColumnRenamed('prediction', 'prediction{}'.format(i))

In [118]:
l[18].show()

+--------+------------+
|movie_id|prediction18|
+--------+------------+
| 1335380|         1.0|
|29062594|         0.0|
| 9252321|         0.0|
|13455076|         0.0|
|24165951|         0.0|
| 1925869|         1.0|
|10799612|         0.0|
|28238240|         0.0|
|17124781|         0.0|
|28207941|         0.0|
|19174305|         0.0|
|18392317|         0.0|
|34420857|         0.0|
| 4039635|         0.0|
| 8034072|         0.0|
| 4016437|         0.0|
| 1520023|         0.0|
|24589422|         1.0|
|35068740|         0.0|
|21132951|         0.0|
+--------+------------+
only showing top 20 rows



In [0]:
joined_df = l[0]
for i in range(1,len(l)):
    joined_df = joined_df.join(l[i], on =['movie_id'], how = 'inner' )

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
cols = []
for i in range(len(l)):
    cols.append('prediction{}'.format(i))

In [0]:
vecAssembler = VectorAssembler(inputCols = cols, outputCol = 'predictions')
joined_df = vecAssembler.transform(joined_df)

In [123]:
joined_df.head(5)

[Row(movie_id='1335380', prediction0=0.0, prediction1=0.0, prediction2=1.0, prediction3=1.0, prediction4=0.0, prediction5=0.0, prediction6=1.0, prediction7=1.0, prediction8=1.0, prediction9=0.0, prediction10=0.0, prediction11=1.0, prediction12=0.0, prediction13=0.0, prediction14=0.0, prediction15=1.0, prediction16=0.0, prediction17=1.0, prediction18=1.0, prediction19=0.0, predictions=SparseVector(20, {2: 1.0, 3: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 11: 1.0, 15: 1.0, 17: 1.0, 18: 1.0})),
 Row(movie_id='29062594', prediction0=1.0, prediction1=1.0, prediction2=1.0, prediction3=0.0, prediction4=0.0, prediction5=0.0, prediction6=0.0, prediction7=0.0, prediction8=1.0, prediction9=0.0, prediction10=0.0, prediction11=0.0, prediction12=0.0, prediction13=0.0, prediction14=0.0, prediction15=1.0, prediction16=1.0, prediction17=0.0, prediction18=0.0, prediction19=0.0, predictions=SparseVector(20, {0: 1.0, 1: 1.0, 2: 1.0, 8: 1.0, 15: 1.0, 16: 1.0})),
 Row(movie_id='9252321', prediction0=1.0, prediction1=1.0

In [0]:
def get_prediction_string(prediction):
    predictions = ""
    for i in range(len(prediction)):
        predictions += str(int(prediction[i]))+" "
    predictions = predictions.rstrip(" ")
    return predictions

In [0]:
from pyspark.sql import types
udffunc = udf(get_prediction_string, types.StringType())
new_df = joined_df.withColumn('predictions', udffunc('predictions'))

In [126]:
new_df.show()

+--------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+--------------------+
|movie_id|prediction0|prediction1|prediction2|prediction3|prediction4|prediction5|prediction6|prediction7|prediction8|prediction9|prediction10|prediction11|prediction12|prediction13|prediction14|prediction15|prediction16|prediction17|prediction18|prediction19|         predictions|
+--------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+--------------------+
| 1335380|        0.0|        0.0|        1.0|        1.0|        0.0|        0.0|        1.0|        1.0|        1.0|        0.0|         0.0|         1.

In [0]:

drop_cols = []
for i in range(len(l)):
    drop_cols.append('prediction{}'.format(i))

In [128]:
new_df = new_df.drop(*drop_cols)
new_df.show()

+--------+--------------------+
|movie_id|         predictions|
+--------+--------------------+
| 1335380|0 0 1 1 0 0 1 1 1...|
|29062594|1 1 1 0 0 0 0 0 1...|
| 9252321|1 1 1 0 0 0 0 0 1...|
|13455076|1 1 1 0 0 0 0 0 1...|
|24165951|1 1 1 0 0 0 0 0 1...|
| 1925869|1 1 1 1 0 1 0 0 0...|
|10799612|1 1 1 0 0 0 0 0 1...|
|28238240|1 1 1 0 0 0 0 0 1...|
|17124781|1 1 1 0 0 0 0 0 1...|
|28207941|1 1 1 0 0 0 0 0 1...|
|19174305|1 1 1 0 0 0 0 0 1...|
|18392317|1 1 1 0 0 0 0 0 1...|
|34420857|1 1 1 0 0 0 0 0 1...|
| 4039635|1 0 1 1 0 0 1 1 1...|
| 8034072|1 1 1 0 0 0 0 0 1...|
| 4016437|1 1 1 0 0 0 0 0 1...|
| 1520023|1 1 1 0 0 0 0 0 1...|
|24589422|1 1 1 1 0 1 0 0 0...|
|35068740|1 1 1 0 0 0 0 0 1...|
|21132951|1 1 1 0 0 0 0 0 1...|
+--------+--------------------+
only showing top 20 rows



In [0]:
new_df.select('*').toPandas().to_csv("/content/gdrive/My Drive/Colab Notebooks/submission.csv", index = False)